In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('data.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")

45 documents
29 classes ['No', 'anxiety', 'appointment_booking', 'cough', 'depressed', 'diet', 'dizzy', 'fever', 'fit', 'goodbye', 'goodbye_2', 'greeting', 'greeting_1', 'greeting_2', 'headache', 'health', 'healthy', 'negative', 'ok', 'provide_location_1', 'provide_location_2', 'provide_location_3', 'runny nose', 'sore_throat', 'stomach', 'stress', 'thanks', 'thanks_1', 'thanks_2']
62 unique lemmatized words ['a', 'am', 'an', 'anxiety', 'appoinment', 'badlapur', 'balanced', 'bhandup', 'book', 'by', 'can', 'cough', 'day', 'deal', 'depressed', 'diet', 'dizzy', 'do', 'eat', 'feeling', 'fever', 'fit', 'for', 'give', 'good', 'have', 'headache', 'health', 'healthy', 'hello', 'hi', 'how', 'hurt', 'i', 'karjat', 'later', 'lot', 'me', 'my', 'negative', 'no', 'nose', 'ok', 'runny', 'see', 'should', 'some', 'sore', 'sore_throat', 'stay', 'staying', 'stomach', 'stress', 'thank', 'thanks', 'thought', 'throat', 'tip', 'what', 'with', 'yes', 'you']


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (45, 2) + inhomogeneous part.

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import random

# Load data
data_file = open('data.json').read()
intents = json.loads(data_file)

words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Process intents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Debugging print to check lengths of input and output vectors
for t in training:
    print(f'Input vector length: {len(t[0])}, Output vector length: {len(t[1])}')

# Shuffle the training data
random.shuffle(training)

# Ensure all lists in training data have consistent lengths before converting to numpy arrays
train_x = []
train_y = []
for t in training:
    train_x.append(t[0])
    train_y.append(t[1])

# Convert lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")

# Create the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and save the model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)

print("model created")


45 documents
29 classes ['No', 'anxiety', 'appointment_booking', 'cough', 'depressed', 'diet', 'dizzy', 'fever', 'fit', 'goodbye', 'goodbye_2', 'greeting', 'greeting_1', 'greeting_2', 'headache', 'health', 'healthy', 'negative', 'ok', 'provide_location_1', 'provide_location_2', 'provide_location_3', 'runny nose', 'sore_throat', 'stomach', 'stress', 'thanks', 'thanks_1', 'thanks_2']
62 unique lemmatized words ['a', 'am', 'an', 'anxiety', 'appoinment', 'badlapur', 'balanced', 'bhandup', 'book', 'by', 'can', 'cough', 'day', 'deal', 'depressed', 'diet', 'dizzy', 'do', 'eat', 'feeling', 'fever', 'fit', 'for', 'give', 'good', 'have', 'headache', 'health', 'healthy', 'hello', 'hi', 'how', 'hurt', 'i', 'karjat', 'later', 'lot', 'me', 'my', 'negative', 'no', 'nose', 'ok', 'runny', 'see', 'should', 'some', 'sore', 'sore_throat', 'stay', 'staying', 'stomach', 'stress', 'thank', 'thanks', 'thought', 'throat', 'tip', 'what', 'with', 'yes', 'you']
Input vector length: 62, Output vector length: 29
In

C:\Users\Windows\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Windows\AppData\Roaming\Python\Python312\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.0502 - loss: 3.3531 
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0388 - loss: 3.4045   
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0727 - loss: 3.3415   
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1242 - loss: 3.2850
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0114 - loss: 3.2856    
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0635 - loss: 3.1298    
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0474 - loss: 3.2502    
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1469 - loss: 3.1665  
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1325 - loss: 3.1784  
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1699 - loss: 3.0783
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1141 - loss: 3.0812  
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/ste

model created


In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
import random

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents data
data_file = open('data.json').read()
intents = json.loads(data_file)

words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Process intents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Debugging print to check lengths of input and output vectors
for t in training:
    print(f'Input vector length: {len(t[0])}, Output vector length: {len(t[1])}')

# Shuffle the training data
random.shuffle(training)

# Ensure all lists in training data have consistent lengths before converting to numpy arrays
train_x = []
train_y = []
for t in training:
    train_x.append(t[0])
    train_y.append(t[1])

# Convert lists to numpy arrays
train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

print("Training data created")

# Debugging print to check shapes of train_x and train_y
print(f'train_x shape: {train_x.shape}')
print(f'train_y shape: {train_y.shape}')

# Create the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and save the model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('model.h5')

print("model created")


45 documents
29 classes ['No', 'anxiety', 'appointment_booking', 'cough', 'depressed', 'diet', 'dizzy', 'fever', 'fit', 'goodbye', 'goodbye_2', 'greeting', 'greeting_1', 'greeting_2', 'headache', 'health', 'healthy', 'negative', 'ok', 'provide_location_1', 'provide_location_2', 'provide_location_3', 'runny nose', 'sore_throat', 'stomach', 'stress', 'thanks', 'thanks_1', 'thanks_2']
62 unique lemmatized words ['a', 'am', 'an', 'anxiety', 'appoinment', 'badlapur', 'balanced', 'bhandup', 'book', 'by', 'can', 'cough', 'day', 'deal', 'depressed', 'diet', 'dizzy', 'do', 'eat', 'feeling', 'fever', 'fit', 'for', 'give', 'good', 'have', 'headache', 'health', 'healthy', 'hello', 'hi', 'how', 'hurt', 'i', 'karjat', 'later', 'lot', 'me', 'my', 'negative', 'no', 'nose', 'ok', 'runny', 'see', 'should', 'some', 'sore', 'sore_throat', 'stay', 'staying', 'stomach', 'stress', 'thank', 'thanks', 'thought', 'throat', 'tip', 'what', 'with', 'yes', 'you']
Input vector length: 62, Output vector length: 29
In

model created


In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import legacy as optimizers
import random

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents data
data_file = open('data.json').read()
intents = json.loads(data_file)

words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Process intents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Debugging print to check lengths of input and output vectors
for t in training:
    print(f'Input vector length: {len(t[0])}, Output vector length: {len(t[1])}')

# Shuffle the training data
random.shuffle(training)

# Ensure all lists in training data have consistent lengths before converting to numpy arrays
train_x = []
train_y = []
for t in training:
    train_x.append(t[0])
    train_y.append(t[1])

# Convert lists to numpy arrays
train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

print("Training data created")

# Debugging print to check shapes of train_x and train_y
print(f'train_x shape: {train_x.shape}')
print(f'train_y shape: {train_y.shape}')

# Create the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model using the legacy optimizer
sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit and save the model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('model.h5')

print("model created")


45 documents
29 classes ['No', 'anxiety', 'appointment_booking', 'cough', 'depressed', 'diet', 'dizzy', 'fever', 'fit', 'goodbye', 'goodbye_2', 'greeting', 'greeting_1', 'greeting_2', 'headache', 'health', 'healthy', 'negative', 'ok', 'provide_location_1', 'provide_location_2', 'provide_location_3', 'runny nose', 'sore_throat', 'stomach', 'stress', 'thanks', 'thanks_1', 'thanks_2']
62 unique lemmatized words ['a', 'am', 'an', 'anxiety', 'appoinment', 'badlapur', 'balanced', 'bhandup', 'book', 'by', 'can', 'cough', 'day', 'deal', 'depressed', 'diet', 'dizzy', 'do', 'eat', 'feeling', 'fever', 'fit', 'for', 'give', 'good', 'have', 'headache', 'health', 'healthy', 'hello', 'hi', 'how', 'hurt', 'i', 'karjat', 'later', 'lot', 'me', 'my', 'negative', 'no', 'nose', 'ok', 'runny', 'see', 'should', 'some', 'sore', 'sore_throat', 'stay', 'staying', 'stomach', 'stress', 'thank', 'thanks', 'thought', 'throat', 'tip', 'what', 'with', 'yes', 'you']
Input vector length: 62, Output vector length: 29
In

ImportError: `keras.optimizers.legacy` is not supported in Keras 3. When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy` optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable `TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.